In [ ]:
import pandas as pd
import numpy as np
from pulp import *
import scipy
from sklearn.cluster import AgglomerativeClustering
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt 

from layout_utils.layout import Layout
from layout_utils.simulation import Order, OrderPickEvent, run_simulation, Agent, s_shape, sim_loop
from utils import get_products
from copy import deepcopy
import os

In [58]:
"""Use this code to concat all orders to generate a large order set"""
orders = []
# Folder containing your text files
folder_path = 'data/orders/large'

# Get a list of all text files in the folder
file_list = [f for f in os.listdir(folder_path)]

# Initialize an empty DataFrame to store the data
combined_data = pd.DataFrame()

# Iterate through each text file and read its contents into a DataFrame
for file_name in file_list:
    file_path = os.path.join(folder_path, file_name)
    with open(file_path, 'r') as file:
        lines = file.readlines()

    for line in lines[2:]:
        orderline = []
        items = line.strip().split()[1:]
        for i in items[::2]:
            orderline.append(int(i))
        orders.append(orderline)

In [59]:
products = get_products(orders)

In [60]:
products_mapping = {p: i for i, p in enumerate(products)}

In [61]:
new_order = []
for order in orders: 
    orderline = []
    for p in order:
        orderline.append(products_mapping[p])
    new_order.append(orderline)

orders = deepcopy(new_order)

In [62]:
product_frequency = {i : 0 for i in products_mapping.values()}
for order in orders:
    for p in order:
        product_frequency[p] += 1

In [63]:
products_srtd = dict(sorted(product_frequency.items(), key=lambda item: item[1], reverse=True))

In [64]:
data = []
for order_id, order in enumerate(orders, start=1):
    data.extend([{'OrderID': order_id, 'Item': item} for item in order])

# Create a pandas DataFrame
df_orders = pd.DataFrame(data)

In [65]:
df_orders

,OrderID,Item
0,1,0
1,1,1
2,1,2
3,1,3
4,1,4
...,...,...
40468,4024,35
40469,4024,26
40470,4024,36
40471,4024,37


In [66]:
warehouse = Layout(40, 40, 1, True, False, None) # two rows reserved for walking
double_deep = "True"
path = os.path.join("data/cache/dist_mats/", "dist_mat_" + str(warehouse.layout_grid.shape) + "_" + str(double_deep) + ".npy")
if os.path.exists(path):
    dist_mat = np.load(path)
else:
    dist_mat = warehouse.gen_dist_mat(path)

In [67]:
df_pivot = df_orders.groupby(['Item', 'OrderID']).aggregate({'Item':'count'})
df_pivot = df_pivot.unstack().fillna(0)

X = np.asmatrix(df_pivot.values)
sX = scipy.sparse.csr_matrix(X) 

print(X)
print(f"No. of products: {X.shape[0]}")
print(f"No. of orders {X.shape[1]}") 

numerator = sX.dot(sX.T)

ones = np.ones(sX.shape[::-1])
B = sX.dot(ones)
denominator = B + B.T - numerator

jaccard = (numerator / (denominator + 0.00001)) 
print(jaccard)

[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
No. of products: 946
No. of orders 4024
  (0, 892)	0.006410255999342564
  (0, 891)	0.006410255999342564
  (0, 890)	0.006410255999342564
  (0, 889)	0.006410255999342564
  (0, 638)	0.005714285387755121
  (0, 341)	0.005494505192609605
  (0, 316)	0.005376343796970763
  (0, 311)	0.005376343796970763
  (0, 140)	0.0040650404851609555
  (0, 433)	0.08974358399079589
  (0, 432)	0.08974358399079589
  (0, 431)	0.08974358399079589
  (0, 430)	0.08974358399079589
  (0, 429)	0.08974358399079589
  (0, 428)	0.08974358399079589
  (0, 427)	0.08917196884254974
  (0, 426)	0.08974358399079589
  (0, 425)	0.08974358399079589
  (0, 424)	0.08974358399079589
  (0, 82)	0.6025640639382009
  (0, 81)	0.6025640639382009
  (0, 80)	0.38367345372761413
  (0, 79)	0.5251396354670594
  (0, 78)	0.5911949313713879
  (0, 17)	0.44062498623046925
  :	:
  (943, 943)	0.99999000

In [68]:
jaccard.A

array([[0.99999994, 0.9038461 , 0.8867924 , ..., 0.        , 0.        ,
        0.        ],
       [0.9038461 , 0.99999993, 0.9791666 , ..., 0.        , 0.        ,
        0.        ],
       [0.8867924 , 0.9791666 , 0.99999993, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.99999   , 0.99999   ,
        0.99999   ],
       [0.        , 0.        , 0.        , ..., 0.99999   , 0.99999   ,
        0.99999   ],
       [0.        , 0.        , 0.        , ..., 0.99999   , 0.99999   ,
        0.99999   ]])

In [69]:
np.asarray(jaccard)

array(<946x946 sparse matrix of type '<class 'numpy.float64'>'
	with 12876 stored elements in COOrdinate format>, dtype=object)

In [70]:
depot = "0,0,0"
depot_temp = depot.split(",")
depot_temp = [int(i) for i in depot_temp]
depot = tuple(depot_temp)
depot

(0, 0, 0)

In [71]:
for n_cluster in range(1, 647):    
    results = []
    clustering = AgglomerativeClustering(n_clusters=n_cluster, metric='precomputed', linkage='complete')
    cluster_labels = clustering.fit_predict(jaccard.A)

    products_srtd_key = {key: value for key, value in sorted(products_srtd.items())}
    df_cluster = pd.DataFrame({"cluster":cluster_labels, "product":products_mapping.values(), "frequency": products_srtd_key.values()})

    n_storage_locs = len(warehouse.storage_locs)
    #n_products = len(warehouse.storage_locs)
    n_products = len(products)

    n_cluster = df_cluster["cluster"].nunique()
    cluster = [i for i in range(n_cluster + 1)]
    storage_locs = [warehouse.nodes_list.index(loc) for loc in warehouse.storage_locs]

    cluster_freq = list(df_cluster.groupby(by="cluster").agg({"frequency":"mean"})["frequency"])

    depot = "0,0,0"
    depot_temp = depot.split(",")
    depot_temp = [int(i) for i in depot_temp]
    depot = tuple(depot_temp)
    depot

    products_in_cluster = list(df_cluster.groupby(by="cluster").agg({"product":"count"})["product"])

    empty_locs = len(storage_locs) - n_products
    cluster_freq.append(0)
    products_in_cluster.append(empty_locs)

    prob = LpProblem("SLAP", LpMinimize)
    y_i_k = LpVariable.dicts("Lagerplatz i in Cluster k", (storage_locs, cluster), 0, 1, LpBinary)

    prob += lpSum(y_i_k[i][k] * dist_mat[depot[0], i] * cluster_freq[k] for i in storage_locs for k in cluster)

    for i in storage_locs:
        prob += lpSum(y_i_k[i][k] for k in cluster) == 1

    for k in cluster:
        prob += lpSum(y_i_k[i][k] for i in storage_locs) == products_in_cluster[k]

    prob.solve(PULP_CBC_CMD(msg=0))

    individual = [0 for i in range(n_products)]
    cluster_assignment = deepcopy(warehouse.layout_grid)

    df_cluster = df_cluster.sort_values(by=["cluster", "frequency"], ascending=[True, False])
    products_to_assign = []
    for k in df_cluster["cluster"].unique():
        products_to_assign.append(list(df_cluster[df_cluster["cluster"] == k]["product"]))

    th = 0.001
    locs_assigned = 0
    location_cluster_mapping = {k : [] for k in range(len(cluster))}
    for i in storage_locs:
        for loc, k in enumerate(cluster):
            if y_i_k[i][k].varValue > th:
                locs_assigned += 1
                cluster_assignment[warehouse.nodes_list[i]] = k 
                # if k in df_cluster["cluster"].unique():
                #     individual[products_to_assign[k].pop(0)] = i
                location_cluster_mapping[k].append(i) 

    for k in range(len(cluster)-1):
        storage_mapping = {}
        distances = dist_mat[0][location_cluster_mapping[k]]
        for idx, dist in zip(location_cluster_mapping[k], distances):
            storage_mapping[idx] = dist
        
        storage_dist_srtd = dict(sorted(storage_mapping.items(), key=lambda item: item[1]))
        storage_keys = list(storage_dist_srtd.keys())
        for i, product in enumerate(products_to_assign[k]):
            individual[product] = storage_keys[i]

    warehouse.storage_assignment = individual 

    route, line, distance, zugriff = sim_loop("stichgang1", orders, warehouse, depot)
    # orders_sim = []
    # for order in orders:
    #     order_s_shaped = s_shape(order, warehouse, depot)
    #     orders_sim.append(order_s_shaped)

    # distance = 0
    # for order in orders_sim:
    #     source = (0,19,0)
    #     for sku in order:
    #         loc = warehouse.nodes_list[warehouse.storage_assignment[sku]] 
    #         distance += warehouse.dist_mat[warehouse.nodes_list.index(source)][warehouse.nodes_list.index(loc)]
    #         source = loc
    #     distance += warehouse.dist_mat[warehouse.nodes_list.index(source)][warehouse.nodes_list.index((0,19,0))]

    print(n_cluster, distance)
    results.append(distance)
    

1 801354.0
2 966284.0
3 1160580.0
4 1202876.0
5 1254058.0
6 1297236.0


KeyboardInterrupt: 

In [72]:
results = []
n_cluster = 1
clustering = AgglomerativeClustering(n_clusters=n_cluster, metric='precomputed', linkage='complete')
cluster_labels = clustering.fit_predict(jaccard.A)

products_srtd_key = {key: value for key, value in sorted(products_srtd.items())}
df_cluster = pd.DataFrame({"cluster":cluster_labels, "product":products_mapping.values(), "frequency": products_srtd_key.values()})

n_storage_locs = len(warehouse.storage_locs)
#n_products = len(warehouse.storage_locs)
n_products = len(products)

n_cluster = df_cluster["cluster"].nunique()
cluster = [i for i in range(n_cluster + 1)]
storage_locs = [warehouse.nodes_list.index(loc) for loc in warehouse.storage_locs]

cluster_freq = list(df_cluster.groupby(by="cluster").agg({"frequency":"mean"})["frequency"])

depot = 26

products_in_cluster = list(df_cluster.groupby(by="cluster").agg({"product":"count"})["product"])

empty_locs = len(storage_locs) - n_products
cluster_freq.append(0)
products_in_cluster.append(empty_locs)

prob = LpProblem("SLAP", LpMinimize)
y_i_k = LpVariable.dicts("Lagerplatz i in Cluster k", (storage_locs, cluster), 0, 1, LpBinary)

prob += lpSum(y_i_k[i][k] * dist_mat[depot, i] * cluster_freq[k] for i in storage_locs for k in cluster)

for i in storage_locs:
    prob += lpSum(y_i_k[i][k] for k in cluster) == 1

for k in cluster:
    prob += lpSum(y_i_k[i][k] for i in storage_locs) == products_in_cluster[k]

prob.solve(PULP_CBC_CMD(msg=0))

individual = [0 for i in range(n_products)]
cluster_assignment = deepcopy(warehouse.layout_grid)

df_cluster = df_cluster.sort_values(by=["cluster", "frequency"], ascending=[True, False])
products_to_assign = []
for k in df_cluster["cluster"].unique():
    products_to_assign.append(list(df_cluster[df_cluster["cluster"] == k]["product"]))

th = 0.001
locs_assigned = 0
location_cluster_mapping = {k : [] for k in range(len(cluster))}
for i in storage_locs:
    for loc, k in enumerate(cluster):
        if y_i_k[i][k].varValue > th:
            locs_assigned += 1
            cluster_assignment[warehouse.nodes_list[i]] = k 
            # if k in df_cluster["cluster"].unique():
            #     individual[products_to_assign[k].pop(0)] = i
            location_cluster_mapping[k].append(i) 

for k in range(len(cluster)-1):
    storage_mapping = {}
    distances = dist_mat[26][location_cluster_mapping[k]]
    for idx, dist in zip(location_cluster_mapping[k], distances):
        storage_mapping[idx] = dist
    
    storage_dist_srtd = dict(sorted(storage_mapping.items(), key=lambda item: item[1]))
    storage_keys = list(storage_dist_srtd.keys())
    for i, product in enumerate(products_to_assign[k]):
        individual[product] = storage_keys[i]

warehouse.storage_assignment = individual 

In [73]:
def render_warehouse(warehouse: Layout, assignment, product_frequency):
    storage_assignment = deepcopy(warehouse.layout_grid)
    walkable_locs = np.where(warehouse.layout_grid == 0)

    for x, y, z in zip(walkable_locs[0], walkable_locs[1], walkable_locs[2]):
        storage_assignment[x,y,z] = -100

    for i, loc in enumerate(assignment):
        storage_assignment[warehouse.nodes_list[loc]] = i 

    storage_frequence = deepcopy(warehouse.layout_grid)
    for i, loc in enumerate(assignment):
        storage_frequence[warehouse.nodes_list[loc]] = product_frequency[i]
    
    node_xyz = np.array([warehouse.pos_dict[v] for v in sorted(warehouse.graph)])
    edge_xyz = np.array([(warehouse.pos_dict[u], warehouse.pos_dict[v]) for u, v in warehouse.graph.edges()])

    data = []
    for x, y, z in zip(node_xyz.T[0], node_xyz.T[1], node_xyz.T[2]):
        data_point = storage_frequence[x,y,z]
        data.append(data_point)

    assignment_plt = []
    for x, y, z in zip(node_xyz.T[0], node_xyz.T[1], node_xyz.T[2]):
        assignment_point = storage_assignment[x,y,z]
        assignment_plt.append(assignment_point)

    trace = go.Scatter3d(customdata=np.stack((data,assignment_plt),axis=-1), hovertemplate = 'x: %{x}<br>y: %{y}<br>z: %{z}<br>Order Frequency: %{customdata[0]}<br>Product: %{customdata[1]}}',
        x = node_xyz.T[0], y = node_xyz.T[1], z = node_xyz.T[2],mode = 'markers', marker = dict(symbol="square",
            size = 12,
            color = data, # set color to an array/list of desired values
            colorscale = 'thermal')
        )
    layout = go.Layout(title = 'Warehouse Visualization')
    fig = go.Figure(data = [trace], layout = layout)
    fig.update_traces(marker_size = 5)
    return fig

In [74]:
fig = render_warehouse(warehouse, warehouse.storage_assignment, product_frequency)

In [75]:
fig.show()